In [1]:
pip install simpy

In [2]:
import simpy
import random

# Constants for the simulation
RANDOM_SEED = 42
SIMULATION_TIME = 1000  # Total simulation time in minutes
INTER_ARRIVAL_TIME = 5  # Average time between vessel arrivals in minutes
CONTAINERS_PER_VESSEL = 7  # Number of containers per vessel
TIME_TO_DISCHARGE_CONTAINER = 3  # Time to discharge one container in minutes
TIME_TO_TRANSPORT_CONTAINER = 6  # Time to transport one container in minutes

# Resources in the terminal
NUMBER_OF_BERTHS = 2
NUMBER_OF_CRANES = 2
NUMBER_OF_TRUCKS = 3

def log_event(env, message):
    """Helper function to log events with the current simulation time."""
    print(f'[{env.now:.2f} minutes] {message}')

class ContainerTerminal:
    def __init__(self, env):
        """Initialize the container terminal with resources and a container queue."""
        self.env = env
        self.berths = simpy.Resource(env, NUMBER_OF_BERTHS)  # Berths for vessels
        self.cranes = simpy.Resource(env, NUMBER_OF_CRANES)  # Cranes for discharging containers
        self.trucks = simpy.Resource(env, NUMBER_OF_TRUCKS)  # Trucks for transporting containers
        self.container_queue = simpy.Store(env)  # Queue to manage containers waiting for trucks

    def discharge_container(self, vessel_id):
        """Process to discharge a container from a vessel."""
        yield self.env.timeout(TIME_TO_DISCHARGE_CONTAINER)  # Time taken to discharge one container
        log_event(self.env, f'Vessel {vessel_id} discharges a container')
        self.container_queue.put(vessel_id)  # Put the discharged container in the queue

    def transport_container(self):
        """Process to transport a container using a truck."""
        vessel_id = yield self.container_queue.get()  # Get a container from the queue
        with self.trucks.request() as truck_request:
            yield truck_request  # Request a truck
            log_event(self.env, f'Truck starts transporting a container for Vessel {vessel_id}')
            yield self.env.timeout(TIME_TO_TRANSPORT_CONTAINER)  # Time taken to transport one container
            log_event(self.env, f'Truck finishes transporting a container for Vessel {vessel_id}')

    def handle_vessel(self, vessel_id):
        """Process to handle the entire vessel operation from arrival to departure."""
        log_event(self.env, f'Vessel {vessel_id} arrives')
        with self.berths.request() as berth_request:
            yield berth_request  # Request a berth for the vessel
            log_event(self.env, f'Vessel {vessel_id} berths')
            for _ in range(CONTAINERS_PER_VESSEL):
                with self.cranes.request() as crane_request:
                    yield crane_request  # Request a crane
                    log_event(self.env, f'Crane starts discharging a container from Vessel {vessel_id}')
                    yield self.env.process(self.discharge_container(vessel_id))  # Discharge a container

                self.env.process(self.transport_container())  # Transport the container

            log_event(self.env, f'Vessel {vessel_id} leaves')

def vessel_generator(env, terminal):
    """Generate vessels arriving at the terminal."""
    vessel_id = 0
    while True:
        yield env.timeout(random.expovariate(1 / INTER_ARRIVAL_TIME) * 60)  # Time until next vessel arrival
        vessel_id += 1
        env.process(terminal.handle_vessel(vessel_id))  # Handle the arriving vessel

def main():
    """Main function to set up and run the simulation."""
    random.seed(RANDOM_SEED)
    env = simpy.Environment()  # Create the SimPy environment
    terminal = ContainerTerminal(env)  # Create the container terminal
    env.process(vessel_generator(env, terminal))  # Start the vessel generator
    env.run(until=SIMULATION_TIME)  # Run the simulation

if __name__ == '__main__':
    main()


[306.02 minutes] Vessel 1 arrives
[306.02 minutes] Vessel 1 berths
[306.02 minutes] Crane starts discharging a container from Vessel 1
[309.02 minutes] Vessel 1 discharges a container
[309.02 minutes] Crane starts discharging a container from Vessel 1
[309.02 minutes] Truck starts transporting a container for Vessel 1
[312.02 minutes] Vessel 1 discharges a container
[312.02 minutes] Crane starts discharging a container from Vessel 1
[312.02 minutes] Truck starts transporting a container for Vessel 1
[313.62 minutes] Vessel 2 arrives
[313.62 minutes] Vessel 2 berths
[313.62 minutes] Crane starts discharging a container from Vessel 2
[315.02 minutes] Truck finishes transporting a container for Vessel 1
[315.02 minutes] Vessel 1 discharges a container
[315.02 minutes] Crane starts discharging a container from Vessel 1
[315.02 minutes] Truck starts transporting a container for Vessel 1
[316.62 minutes] Vessel 2 discharges a container
[316.62 minutes] Crane starts discharging a container fr